# **Codebase for [TentNet](https://ieeexplore.ieee.org/abstract/document/9283377)**

In this notebook I will demonstrate three pre-trained convolutional neural network (CNN) models, created from scratch, on the following two datasets:


1.   https://www.kaggle.com/rhammell/planesnet
2.   https://www.kaggle.com/rhammell/ships-in-satellite-imagery

In addition to this, I will demonstrate theses models on two different synthetic datasets that will be generated by a generative adversarial network (GAN) model from NVIDIA and TensorFlow respectively. This is for the main purpose of this project: identifying tents in urban areas from satellite imagery.




**Please note that a large portion of this code comes from the first deliverable as I am presenting the entire project now.**

> Use these flags to first define the environment this notebook is being ran on


In [ ]:
# Set what I'm doing with this instance of the notebook
# For some reason, the GAN code and the CNN code cannot be ran in the same
# instance due to the GAN code requiring "eager execution" while the CNN
# code requires that not be the case
run_cnn = False

# Set what I'm running this notebook on
colab = True

# Decide to use CPU instead of GPU if desired
use_cpu = False
cpu_cores = 4
cpu_count = 1

> Then run this cell to take care of some initial imports

In [ ]:
# Install a package called 'pillow' to import this
import PIL
from PIL import Image

import json
import numpy as np
from matplotlib import pyplot as plt
import numpy as np
import glob

# Dataset Processing



> First, let's extract the datasets assuming that they're named as "planes.zip" and "ships.zip", respectively, from the following two links:


1.   https://www.kaggle.com/rhammell/planesnet
2.   https://www.kaggle.com/rhammell/ships-in-satellite-imagery



In [ ]:
!unzip -q planes.zip -d planes
!unzip -q ships.zip -d ships

> Next, let's load these datasets with the json library


In [ ]:
# Load the planes dataset's json
path_preceed = '/content/' if colab else ''
with open(path_preceed + 'planes/planesnet.json') as dataset:
    planes_dataset = json.load(dataset)

# Load the ship dataset's json
with open(path_preceed + 'ships/shipsnet.json') as dataset:
    ships_dataset = json.load(dataset)



> Finally, let's view an image from each dataset using sample code from the datasets' authors on Kaggle



In [ ]:
# First, show a plane
# https://www.kaggle.com/rhammell/displaying-and-saving-image-chips
plane = np.array(planes_dataset['data'][22]).astype('uint8')
plane = plane.reshape((3, 400)).T.reshape((20, 20, 3))
plt.imshow(plane)
print("Here's an example plane from the dataset:")
plt.show()

# Second, show a ship
# https://www.kaggle.com/rhammell/reforming-and-displaying-image-chips
ship = np.array(ships_dataset['data'][22]).astype('uint8')
ship = ship.reshape((3, 6400)).T.reshape((80, 80, 3))
plt.imshow(ship)
print("\nHere's an example ship from the dataset:")
plt.show()

# Model Creation



> First, let's import the libraries we'll need to make the CNN models and set the seed
*   The code for setting the seeds came from the Keras documentation here: https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development







In [ ]:
# Hide warnings and logging
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

if colab:
  %tensorflow_version 1.x
import tensorflow as tf

if not run_cnn:
  tf.enable_eager_execution()
  tf.config.experimental_run_functions_eagerly(True)

tf.get_logger().setLevel("ERROR")

# Import the modules needed from Keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Flatten, Dropout, BatchNormalization
from keras.layers import LeakyReLU, ReLU, PReLU, Softmax, ZeroPadding2D
from keras.optimizers import Adagrad, Adam, SGD, RMSprop, Nadam
from keras import regularizers 

# Set the seeds
import random as rn
np.random.seed(2020)
rn.seed(2020)
tf.set_random_seed(2020)

# Set the tensorflow configuration
from keras import backend as K
from keras.backend.tensorflow_backend import set_session  
config = tf.ConfigProto(log_device_placement=True)

# Set which device to use
if use_cpu:
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

    config = tf.ConfigProto(
        intra_op_parallelism_threads=cpu_cores,
        inter_op_parallelism_threads=cpu_cores, 
        allow_soft_placement=True,
        device_count = {'CPU' : cpu_count,
                        'GPU' : 0},
        log_device_placement=True
    )
else:
    config = tf.ConfigProto(
        log_device_placement=True
    )
    
    # Optimize my personal GPU if not on Colab
    if not colab:
        config.gpu_options.allow_growth=True

sess = tf.Session(config=config)

if run_cnn:
  K.set_session(sess)



> **Next, let's define the method to create the first model**
*   This transfers learning from the ResNet50V2 model in Keras





In [ ]:
from keras.applications.resnet_v2 import ResNet50V2

def create_cnn_model_one(width, height, depth, outputs = 2, 
                     optimizer = Adagrad, activation = ReLU, 
                     learning_rate = 0.001):
    # Initialize the model
    model = Sequential()
    pre_trained = ResNet50V2(weights='imagenet', include_top=False,
                                    input_shape=(width, height, depth))

    # Add a flatten layer
    model.add(Flatten())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(256,
                    activation = 'relu',
                    kernel_initializer='random_normal'))
    
    # Add a dropout layer
    model.add(Dropout(0.4))

    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(128,
                    activation = 'relu',
                    kernel_initializer='random_normal'))
    
    # Add a dropout layer
    model.add(Dropout(0.3))
    
    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(64,
                    activation = 'relu',
                    kernel_initializer='random_normal'))
    
    # Add a dropout layer
    model.add(Dropout(0.2))
    
    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(16,
                    activation = 'relu',
                    kernel_initializer='random_normal'))

    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Finally, add the output layer
    model.add(Dense(outputs,
                    activation = 'softmax',
                    kernel_initializer='random_normal'))

    # Compile the model
    complete_model = Sequential()
    complete_model.add(pre_trained)
    complete_model.add(model)
    complete_model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizer(lr=learning_rate),
                  metrics = ['accuracy'])

    complete_model.name = "TentNetwork_ResNet50V2"
    return complete_model



> **Next, let's define the method to create the second model**
*   This transfers learning from the InceptionV3 model in Keras



In [ ]:
from keras.applications.inception_v3 import InceptionV3

def create_cnn_model_two(width, height, depth, outputs = 2, 
                     optimizer = Adagrad, activation = ReLU, 
                     learning_rate = 0.001):
    # Initialize the model
    model = Sequential()
    pre_trained = InceptionV3(weights='imagenet', include_top=False,
                              input_shape=(width, height, depth))

    # Add a flatten layer
    model.add(Flatten())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(256,
                    activation = 'relu',
                    kernel_initializer='random_normal'))
    
    # Add a dropout layer
    model.add(Dropout(0.4))

    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(128,
                    activation = 'relu',
                    kernel_initializer='random_normal'))
    
    # Add a dropout layer
    model.add(Dropout(0.3))
    
    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(64,
                    activation = 'relu',
                    kernel_initializer='random_normal'))
    
    # Add a dropout layer
    model.add(Dropout(0.2))
    
    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(16,
                    activation = 'relu',
                    kernel_initializer='random_normal'))

    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Finally, add the output layer
    model.add(Dense(outputs,
                    activation = 'softmax',
                    kernel_initializer='random_normal'))

    # Compile the model
    complete_model = Sequential()
    complete_model.add(pre_trained)
    complete_model.add(model)
    complete_model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizer(lr=learning_rate),
                  metrics = ['accuracy'])

    complete_model.name = "TentNetwork_InceptionV3"
    return complete_model



> **Next, let's define the method to create the third model**
*   This transfers learning from the MobileNet2 model in Keras



In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2

def create_cnn_model_three(width, height, depth, outputs = 2, 
                     optimizer = Adagrad, activation = ReLU, 
                     learning_rate = 0.001):
    # Initialize the model
    model = Sequential()
    pre_trained = MobileNetV2(weights='imagenet', include_top=False,
                              input_shape=(width, height, depth))

    # Add a flatten layer
    model.add(Flatten())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(256,
                    activation = 'relu',
                    kernel_initializer='random_normal'))
    
    # Add a dropout layer
    model.add(Dropout(0.4))

    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(128,
                    activation = 'relu',
                    kernel_initializer='random_normal'))
    
    # Add a dropout layer
    model.add(Dropout(0.3))
    
    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(64,
                    activation = 'relu',
                    kernel_initializer='random_normal'))
    
    # Add a dropout layer
    model.add(Dropout(0.2))
    
    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Add a dense layer
    model.add(Dense(16,
                    activation = 'relu',
                    kernel_initializer='random_normal'))

    # Add a batch normalization
    model.add(BatchNormalization())

    # Add activation layer
    model.add(activation())

    # Finally, add the output layer
    model.add(Dense(outputs,
                    activation = 'softmax',
                    kernel_initializer='random_normal'))

    # Compile the model
    complete_model = Sequential()
    complete_model.add(pre_trained)
    complete_model.add(model)
    complete_model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizer(lr=learning_rate),
                  metrics = ['accuracy'])

    complete_model.name = "TentNetwork_MobileNetV2"
    return complete_model

> Next, we will define a method that uses cross-fold validation to evaluate the model using data augmentation (code from the Keras documentation) and return the result

In [ ]:
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import class_weight
import numpy as np

kf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 2020)

def validate_model(folds, inputs, outputs, outputs_indiv, params = None, epochs = 20, 
                   verbose=True, create_cnn_model = None):
    # Initialize variables to keep track of the results
    overall_train_acc = 0
    overall_train_loss = 0
    overall_test_acc = 0
    overall_test_loss = 0
    count = 0

    # Cycle through the folds
    total_start = time.time()
    for train_index, test_index in folds:
        start = time.time()

        count += 1
        print("\nEvaluating fold " + str(count) + "/" + str(len(folds)), end="")

        # Split the data
        X_train, X_test = inputs[train_index], inputs[test_index]
        Y_train, Y_test = outputs[train_index], outputs[test_index]

        # Initialize a model
        if params == None:
            model = create_cnn_model(len(inputs[0]),
                                  len(inputs[0][0]),
                                  len(inputs[0][0][0]))
        else:
            model = create_cnn_model(**params)

        # Compute the class weights
        weights = class_weight.compute_class_weight('balanced',
                                                    np.unique(outputs_indiv[train_index]), 
                                                    outputs_indiv[train_index])

        # Train and test the model on this fold
        result = model.fit(X_train, Y_train,
                           batch_size = 8,
                           epochs = epochs,
                           validation_data = (X_test, Y_test),
                           class_weight = weights,
                           verbose = 0)

        # Store the accuracy and loss
        overall_train_acc += result.history['acc'][len(result.history['acc']) - 1]
        overall_train_loss += result.history['loss'][len(result.history['loss']) - 1]
        overall_test_acc += result.history['val_acc'][len(result.history['val_acc']) - 1]
        overall_test_loss += result.history['val_loss'][len(result.history['val_loss']) - 1]

        # Print this fold's results if verbose
        end = time.time()
        if(verbose):
            print(": completed in " + str(end - start) + "s" + 
                "\n\tTrain accuracy\t= " + str(result.history['acc'][len(result.history['acc']) - 1]) + 
                "\n\tTrain loss\t= " + str(result.history['loss'][len(result.history['loss']) - 1]) +
                "\n\tTest accuracy\t= " + str(result.history['val_acc'][len(result.history['val_acc']) - 1]) +
                "\n\tTest loss\t= " + str(result.history['val_loss'][len(result.history['val_loss']) - 1]))
  
    # Output and return the result
    overall_train_acc /= count
    overall_train_loss /= count
    overall_test_acc /= count
    overall_test_loss /= count

    total_end = time.time()
    print("\nEvaluation completed in " + str(total_end - total_start) + "s" + 
          "\n\tTrain accuracy\t= " + str(overall_train_acc) + 
          "\n\tTrain loss\t= " + str(overall_train_loss) +
          "\n\tTest accuracy\t= " + str(overall_test_acc) +
          "\n\tTest loss\t= " + str(overall_test_loss))

    return {'train_acc' : overall_train_acc, 
          'train_loss' : overall_train_loss, 
          'test_acc' : overall_test_acc, 
          'test_loss' : overall_test_loss}

> Lastly, let's define a method to perform a grid search using sklearn on this model to determine the best parameters for each dataset before running the validation method previously defined


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
def model_grid_search(classifier, grid, inputs, outputs):
    grid = GridSearchCV(  estimator = classifier,
                          param_grid = grid,
                          n_jobs = -1 if colab else 
                                        (1 if not use_cpu else 
                                             (2 if cpu_count == 1 else 
                                                  cpu_cores)),
                          cv = 2,
                          verbose = 1)
    result = grid.fit(inputs, outputs)
    return result.best_score_, result.best_params_, grid

# Model Evaluation: Planes



> First, let's prepare the dataset by converting each of the images to the author's suggestion as shown before



In [ ]:
# Initialize a list for the converted 
converted_dataset = list()

# Cycle through each instance in the dataset
for plane in planes_dataset['data']:
    plane = np.array(plane).astype('uint8')
    plane = plane.reshape((3, 400)).T.reshape((20, 20, 3))

    # We need to pad the image to make it at least 75 x 75 which is the minmum
    # size for the pre-trained models (makes it 80 x 80)
    plane = np.pad(plane, ((30, 30), (30, 30), (0, 0)),
                   mode='constant', constant_values=3)

    converted_dataset.append(plane)

# Convert the final list into a numpy array
converted_dataset = np.asarray(converted_dataset)

> Next, let's get the outputs and split our dataset using the StratifiedKFold object defined before, and convert the outputs to onehot encoding using Keras' to_categorical method


In [ ]:
# Split the dataset into 10 folds and store the indicies
outputs = np.array(planes_dataset['labels']).astype('uint8')
indicies = list(kf.split(converted_dataset, outputs))

# Convert the outputs to onehot encoding
from keras.utils import to_categorical
outputs_onehot = to_categorical(outputs)

---
**Model 1**



> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_one, verbose=0)

# Define the parameters
width = [plane.shape[0]]
height = [plane.shape[1]]
depth = [plane.shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_one, best_params_one, _ = model_grid_search(classifier, grid, 
                                             converted_dataset, outputs_onehot)
print("Best score: " + str(best_score_one))
print("Best parameters: " + str(best_params_one))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_one = {'width': plane.shape[0], 'height': plane.shape[1], 'depth': plane.shape[2],
                   'learning_rate': 0.001, 'activation': ReLU, 'optimizer': Adam}
model_one = create_cnn_model_one(**best_params_one)
model_one.summary()

> Finally, let's validate this best accuracy using the method defined before with data augmentation and 10-cross fold validation

In [ ]:
results_one = validate_model(indicies,
                             converted_dataset, outputs_onehot, outputs,
                             best_params_one,
                             create_cnn_model = create_cnn_model_one)
print(results_one)

---
**Model 2**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_two, verbose=0)

# Define the parameters
width = [plane.shape[0]]
height = [plane.shape[1]]
depth = [plane.shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_two, best_params_two, _ = model_grid_search(classifier, grid, 
                                             converted_dataset, outputs_onehot)
print("Best score: " + str(best_score_two))
print("Best parameters: " + str(best_params_two))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_two = {'width': plane.shape[0], 'height': plane.shape[1], 'depth': plane.shape[2],
                   'learning_rate': 0.001, 'activation': Softmax, 'optimizer': Adagrad}
model_two = create_cnn_model_two(**best_params_two)
model_two.summary()

> Finally, let's validate this best accuracy using the method defined before with data augmentation and 10-cross fold validation

In [ ]:
results_two = validate_model(indicies,
                              converted_dataset, outputs_onehot, outputs,
                              best_params_two,
                              create_cnn_model = create_cnn_model_two)
print(results_two)

---
**Model 3**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_three, verbose=0)

# Define the parameters
width = [plane.shape[0]]
height = [plane.shape[1]]
depth = [plane.shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_three, best_params_three, _ = model_grid_search(classifier, grid, 
                                             converted_dataset, outputs_onehot)
print("Best score: " + str(best_score_three))
print("Best parameters: " + str(best_params_three))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_three = {'width': plane.shape[0], 'height': plane.shape[1], 'depth': plane.shape[2],
                   'learning_rate': 0.001, 'activation': ReLU, 'optimizer': Adam}
model_three = create_cnn_model_three(**best_params_three)
model_three.summary()

> Finally, let's validate this best accuracy using the method defined before with data augmentation and 10-cross fold validation

In [ ]:
results_three = validate_model(indicies,
                               converted_dataset, outputs_onehot, outputs,
                               best_params_three,
                               create_cnn_model = create_cnn_model_three)
print(results_three)


# Model Evaluation: Ships



> First, let's prepare the dataset by converting each of the images to the author's suggestion as shown before



In [ ]:
# Initialize a list for the converted 
converted_dataset = list()

# Cycle through each instance in the dataset
for ship in ships_dataset['data']:
    ship = np.array(ship).astype('uint8')
    ship = ship.reshape((3, 6400)).T.reshape((80, 80, 3))
    converted_dataset.append(ship)

# Convert the final list into a numpy array
converted_dataset = np.asarray(converted_dataset)

> Next, let's get the outputs and split our dataset using the StratifiedKFold object defined before, and convert the outputs to onehot encoding using Keras' to_categorical method


In [ ]:
# Split the dataset into 10 folds and store the indicies
outputs = np.array(ships_dataset['labels']).astype('uint8')
indicies = list(kf.split(converted_dataset, outputs))

# Convert the outputs to onehot encoding
from keras.utils import to_categorical
outputs_onehot = to_categorical(outputs)

---
**Model 1**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_one, verbose=0)

# Define the parameters
width = [ship.shape[0]]
height = [ship.shape[1]]
depth = [ship.shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_one, best_params_one, _ = model_grid_search(classifier, grid, 
                                             converted_dataset, outputs_onehot)
print("Best score: " + str(best_score_one))
print("Best parameters: " + str(best_params_one))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_one = {'width': ship.shape[0], 'height': ship.shape[1], 'depth': ship.shape[2],
                   'learning_rate': 0.001, 'activation': Softmax, 'optimizer': Adam}
model_one = create_cnn_model_one(**best_params_one)
model_one.summary()

> Finally, let's validate the accuracy using method defined before with data augmentation and 10-cross fold validation

In [ ]:
results_one = validate_model(indicies,
                             converted_dataset, outputs_onehot, outputs,
                             best_params_one,
                             create_cnn_model = create_cnn_model_one)
print(results_one)

---
**Model 2**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_two, verbose=0)

# Define the parameters
width = [ship.shape[0]]
height = [ship.shape[1]]
depth = [ship.shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_two, best_params_two, _ = model_grid_search(classifier, grid, 
                                             converted_dataset, outputs_onehot)
print("Best score: " + str(best_score_two))
print("Best parameters: " + str(best_params_two))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_two = {'width': ship.shape[0], 'height': ship.shape[1], 'depth': ship.shape[2],
                   'learning_rate': 0.001, 'activation': Softmax, 'optimizer': Adam}
model_two = create_cnn_model_two(**best_params_two)
model_two.summary()

> Finally, let's validate the accuracy using method defined before with data augmentation and 10-cross fold validation

In [ ]:
results_two = validate_model(indicies,
                               converted_dataset, outputs_onehot, outputs,
                               best_params_two,
                               create_cnn_model = create_cnn_model_two)
print(results_two)

---
**Model 3**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_three, verbose=0)

# Define the parameters
width = [ship.shape[0]]
height = [ship.shape[1]]
depth = [ship.shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_three, best_params_three, _ = model_grid_search(classifier, grid, 
                                             converted_dataset, outputs_onehot)
print("Best score: " + str(best_score_three))
print("Best parameters: " + str(best_params_three))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_three = {'width': ship.shape[0], 'height': ship.shape[1], 'depth': ship.shape[2],
                     'learning_rate': 0.001, 'activation': Softmax, 'optimizer': Adam}
model_three = create_cnn_model_three(**best_params_three)
model_three.summary()

> Finally, let's validate the accuracy using method defined before with data augmentation and 10-cross fold validation

In [ ]:
results_three = validate_model(indicies,
                               converted_dataset, outputs_onehot, outputs,
                               best_params_three,
                               create_cnn_model = create_cnn_model_three)
print(results_three)

# xView Dataset

> To train the algorithm on pictures of tents, I will be using a small subset of the [xView dataset](https://arxiv.org/abs/1802.07856). The total size of the data [available to download](https://challenge.xviewdataset.org/data-download) is ~18.2GB so I won't have the option to download it in this notebook but I will show my process to extract the tent/huts images from it then provide a download at the end of this section.

> First, let's read the GEOJSON file that contains all of the features in this dataset and see what the first feature looks like

In [ ]:
with open('Datasets/xView/train_labels/xView_train.geojson') as f:
    xview_labels = json.load(f)

print("Here's what the first feature in the dataset looks like:\n")
print(xview_labels['features'][0])

> Next, let's see what this first feature is

In [ ]:
print("Here's the type of feature for this first entry:\n")
print(xview_labels['features'][0]['properties']['type_id'])
print("\nThis type is referred to as \"Building\"")
print("https://github.com/DIUx-xView/xview2018-baseline/blob/master/xview_class_labels.txt")

> Next, let's see what the filename is that this feature is in

In [ ]:
print("Here's the filename of the first entry:\n")
print(xview_labels['features'][0]['properties']['image_id'])
print("\nNote that the images available from the website don't represent the entire dataset so many are missing")

> Next, let's see how many images in this dataset contain the feature we're after: "Hut/Tent"

In [ ]:
# 71 == hut/tent
tent_files = []
tent_features = []

for feature in xview_labels['features']:
    if feature['properties']['type_id'] == 71:
        if feature['properties']['image_id'] not in tent_files:
            tent_files.append(feature['properties']['image_id'])
        tent_features.append(feature)
        
num_files = len(tent_files)
print("The total number of files with features classified as type \"Hut/Tent\" is " + str(num_files))
print("The total number of features (i.e. actual tents/huts) in the " + str(num_files) + " files is " + str(len(tent_features)))

> Next, out of these files, let's see how many are actually given to us

In [ ]:
file_paths = {}
found_files = []
for file in tent_files:
    res = glob.glob('Datasets\\xView\\**/' + file, recursive=True)
    if len(res) > 0:
        file_paths[file] = res[0]
        found_files.append(file)

num_actual_files = len(found_files)
print("The number of files with tents/huts that actually exist in this download is " + str(num_actual_files) + "/" + str(num_files))

> Then, within these files that are given to us, how many "Hut/Tent" features are there in total

In [ ]:
found_features = []

for feature in tent_features:
    if feature['properties']['image_id'] in found_files:
        found_features.append(feature)
        
print("The number of features in the " + str(num_actual_files) + " files is " + str(len(found_features)))

> With these files, let's figure out what size of image we should create. We'll first look at the largest width/height along with the average

In [ ]:
# First, let's figure out what the largest height and width is
king_width = -1
king_height = -1

avg_width = 0
avg_height = 0
for feature in found_features:
    bound = [int(x.strip()) for x in feature['properties']['bounds_imcoords'].split(',')]
    
    height = bound[3] - bound[1]
    avg_height += height
    if height > king_height:
        king_height = height
    
    width = bound[2] - bound[0]
    avg_width += width
    if width > king_width:
        king_width = width

avg_height /= len(found_features)
avg_width /= len(found_features)
print("Largest width  = " + str(king_width))
print("Largest height = " + str(king_height))
print()
print("Average width  = " + str(avg_width))
print("Average height = " + str(avg_height))

> We can see that the largest dimensions are an anomaly and wouldn't give us a good image to train with for the smaller features. The GAN we'll be using needs the image resolution to be a power of two so let's set the image size to 64 x 64 and discard any images with dimensions larger than 64 - 5 = 59 (the boundaries are tight to the objects so we'll give them a small buffer)

In [ ]:
count = 0
for feature in found_features:
    bound = [int(x.strip()) for x in feature['properties']['bounds_imcoords'].split(',')]
    
    height = bound[3] - bound[1]
    width = bound[2] - bound[0]
    
    if height <= 59 and width <= 59:
        # Change the bounds so it's exactly 64 x 64
        height_diff = 64 - height
        width_diff = 64 - width
        
        height_mod = int(height_diff / 2)
        width_mod = int(width_diff / 2)

        #Change the bounds to match 64 x 64
        tmp = height_mod * 2
        if tmp == height_diff:
            bound[3] += height_mod
            bound[1] -= height_mod
        elif tmp > height_diff:
            bound[3] += height_mod + 1
            bound[1] -= height_mod - (tmp - height_diff) + 1
        elif tmp < height_diff:
            bound[3] += height_mod + 1
            bound[1] -= height_mod - (height_diff - tmp) + 1
        
        tmp = width_mod * 2
        if tmp == width_diff:
            bound[2] += width_mod
            bound[0] -= width_mod
        elif tmp > width_diff:
            bound[2] += width_mod + 1
            bound[0] -= width_mod - (tmp - width_diff) + 1
        elif tmp < width_diff:
            bound[2] += width_mod + 1
            bound[0] -= width_mod - (width_diff - tmp) + 1
        
        # Load the image
        imageObject  = Image.open(file_paths[feature['properties']['image_id']])
        
        # Crop and save it
        cropped = imageObject.crop((bound[0], bound[1], bound[2], bound[3]))
        cropped.save("Datasets/xview_tents/" + str(count) + ".jpg")

        count += 1
        
print("Total images: " + str(count))

> We kept ~89% of the "Hut/Tent" features that were available to us with the 64 x 64 constraint. This output is available on my Google Drive so we will download it to this notebook
*   https://drive.google.com/file/d/1BLWfARXpf4D1n21JbAt6k2VtyEa6M2t4/view?usp=sharing

In [ ]:
!gdown --id 1BLWfARXpf4D1n21JbAt6k2VtyEa6M2t4 --output tents.zip
!unzip -q tents.zip -d tents

# GAN Creation: StyleGAN2

> For the generative adversarial network aspect of this project, I will be using an implementation from NVIDIA Labs called 'StyleGAN2'. This takes some time and I end up generating 1,000 images so I will show my process and provide a download link at the end of this section.
*   https://github.com/NVlabs/stylegan2
*   http://arxiv.org/abs/1912.04958

> Let's clone StyleGAN2's repository and verify the install of NVCC in this notebook (code from the repository's README)

In [ ]:
!git clone https://github.com/NVlabs/stylegan2.git
    
if colab:
    !nvcc /content/stylegan2/test_nvcc.cu -o /content/stylegan2/test_nvcc -run
else:
    !nvcc stylegan2/test_nvcc.cu -o stylegan2/test_nvcc -run

> Next, let's use their 'dataset_tool.py' file to make a TFRecord dataset of the tents dataset for their GAN

In [ ]:
if colab:
    !python /content/stylegan2/dataset_tool.py create_from_images /content/tents/tf/tents/ /content/tents/xview_tents/
else:
    !python stylegan2/dataset_tool.py create_from_images tents/tf/tents/ tents/xview_tents/

> Now let's train the GAN on this dataset using the original StyleGAN
* Note: you need to use GPU processing enabled for this

In [ ]:
if colab:
    !python /content/stylegan2/run_training.py --data-dir=/content/tents/tf/ --dataset=tents --config=config-a --mirror-augment=true --result-dir=/content/tents/result-a/
else:
    !python stylegan2/run_training.py --data-dir=tents/tf/ --dataset=tents --config=config-a --mirror-augment=true --result-dir=tents/result-a/

> Next, let's generate 1,000 images

In [ ]:
if colab:
  !python /content/stylegan2/run_generator.py generate-images --network=/content/tents/result-a/00000-stylegan2-tents-1gpu-config-a/network-snapshot-000001.pkl --seeds=2020-3019 --truncation-psi=1.0
else:
  !python stylegan2/run_generator.py generate-images --network=tents/result-a/00000-stylegan2-tents-1gpu-config-a/network-snapshot-000001.pkl --seeds=2020-3019 --truncation-psi=1.0

> Next, let's train the GAN on the dataset using StyleGAN2
* Note: you need to use GPU processing enabled for this

In [ ]:
if colab:
    !python /content/stylegan2/run_training.py --data-dir=/content/tents/tf/ --dataset=tents --config=config-f --mirror-augment=true --result-dir=/content/tents/result-f/
else:
    !python stylegan2/run_training.py --data-dir=tents/tf/ --dataset=tents --config=config-f --mirror-augment=true --result-dir=tents/result-f/

> Next, let's generate 1,000 images

In [ ]:
if colab:
  !python /content/stylegan2/run_generator.py generate-images --network=/content/tents/result-f/00000-stylegan2-tents-1gpu-config-f/network-snapshot-000000.pkl --seeds=2020-3019 --truncation-psi=1.0
else:
  !python stylegan2/run_generator.py generate-images --network=tents/result-f/00000-stylegan2-tents-1gpu-config-f/network-snapshot-000000.pkl --seeds=2020-3019 --truncation-psi=1.0

> These outputs are available on my Google Drive so we will download them to this notebook
*   https://drive.google.com/file/d/1mak_n_OfgvleUEtu8kp_63ysSsbTVsVT/view?usp=sharing
*   https://drive.google.com/file/d/1Q2_LG5Ea8B45HntTdqxLcOQHoEvpiExF/view?usp=sharing

In [ ]:
!gdown --id 1mak_n_OfgvleUEtu8kp_63ysSsbTVsVT --output output_a.zip
!unzip -q output_a.zip -d output_a

!gdown --id 1Q2_LG5Ea8B45HntTdqxLcOQHoEvpiExF --output output_f.zip
!unzip -q output_f.zip -d output_f

> Let's view an image from the original StyleGAN

In [ ]:
import matplotlib.image as mpimg
if colab:
  img = mpimg.imread('/content/output_a/seed2500.png')
else:
  img = mpimg.imread('output_a/seed2500.png')
  
imgplot = plt.imshow(img)
plt.show()

> Finally, let's view an image from the StyleGAN2

In [ ]:
import matplotlib.image as mpimg
if colab:
  img = mpimg.imread('/content/output_f/seed2020.png')
else:
  img = mpimg.imread('output_f/seed2020.png')
  
imgplot = plt.imshow(img)
plt.show()



> We can see that the output of the StyleGAN2 configuration is much cleaner/smoother than the StyleGAN configuration so we will use it for testing. Regardless, the output is very abstract in both cases. I'm assuming this has to do with the low resolution of the images I fed to it, as well as the small size of the dataset it learned from. Regardless, we will see how the model performs when training on these images and testing on the actual images. To assist further, all images will be converted to grayscale so the colors don't have a great impact when the model is training.



# GAN Creation: DCGAN from TensorFlow

> To provide a comparison, I will use the Deep Convolutional Generative Adversarial Network (DCGAN) implementation from TensorFlow:
*   https://www.tensorflow.org/tutorials/generative/dcgan

> The code from this resource will be pasted below but instead of using the *exact* implementation for the model portion, I will make one based off of it with parameters that can be changed for the purpose of running a grid search to see how it affects the output!
*   We will use the summation of the generator and discriminator loss as the measure of performance for this






> First, let's grab the imports they use


In [ ]:
from tensorflow.keras import layers, losses, optimizers

> Next, let's adapt their generator model method so we can run a grid search on it

In [ ]:
def make_generator_model(width, height, depth,
                         activation = layers.LeakyReLU, use_bias = False,
                         optimizer = Adam, learning_rate = 0.001):
    model = tf.keras.Sequential()
    model.add(layers.Dense(int(width / 4)*int(height / 4)*256, use_bias=use_bias, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(activation())

    model.add(layers.Reshape((int(width / 4), int(height / 4), 256)))

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=use_bias))
    model.add(layers.BatchNormalization())
    model.add(activation())
    
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=use_bias))
    model.add(layers.BatchNormalization())
    model.add(activation())

    model.add(layers.Conv2DTranspose(depth, (5, 5), strides=(2, 2), padding='same', use_bias=use_bias, activation='tanh'))

    optim = optimizer(learning_rate)
    return model, optim

> Next, let's adapt their discriminator model method so we can run a grid search on it

In [ ]:
def make_discriminator_model(width, height, depth,
                             activation = layers.LeakyReLU, dropout  = 0.3,
                             optimizer = Adam, learning_rate = 0.001):
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[width, height, depth]))
    model.add(activation())
    model.add(layers.Dropout(dropout))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(activation())
    model.add(layers.Dropout(dropout))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    optim = optimizer(learning_rate)
    return model, optim

> Next, let's use their loss method for the discriminator


In [ ]:
def discriminator_loss(real_output, fake_output, cross_entropy):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

> Next, let's use their loss method for the generator


In [ ]:
def generator_loss(fake_output, cross_entropy):
    return cross_entropy(tf.ones_like(fake_output), fake_output)



> Next, let's use their step method for training, modified to allow for a grid search



In [ ]:
@tf.function
def train_step(images, loss,
               generator, discriminator,
               generator_optimizer, discriminator_optimizer):

    noise = tf.random.normal([len(images), 100])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output, loss)
        disc_loss = discriminator_loss(real_output, fake_output, loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss



> Next, let's use adapt method for training the model for our grid search



In [ ]:
import tensorflow.contrib.eager as tfe
def train(dataset, epochs, loss,
          generator, discriminator,
          generator_optimizer, discriminator_optimizer):
    for epoch in range(epochs):
        # Need this for minor difference in TensorFlow 1.X between my machines
        try:
            for image_batch in tfe.Iterator(dataset):
                gen_loss, disc_loss = train_step(image_batch, loss,
                                                 generator, discriminator,
                                                 generator_optimizer, discriminator_optimizer)
        except:
            for image_batch in dataset.__iter__():
                gen_loss, disc_loss = train_step(image_batch, loss,
                                                 generator, discriminator,
                                                 generator_optimizer, discriminator_optimizer)
            
    # Return the end result, total loss
    return (gen_loss + disc_loss)



> Next, let's define a custom grid search method for the GAN that's similar to GridSearchCV



In [ ]:
from sklearn.model_selection import ParameterGrid

def gan_grid_search(generator_method, discriminator_method,
                    generator_grid, discriminator_grid, 
                    dataset, epochs = 10):
    # Convert grids to parameter grids for the generator and discriminator
    generator_grid = list(ParameterGrid(generator_grid))
    discriminator_grid = list(ParameterGrid(discriminator_grid))
    
    # Output how many models we'll be testing
    total = len(generator_grid) * len(discriminator_grid)
    print("Fitting "+ str(total) + " candidates")
    
    # Begin testing the models
    count = 0
    best_params = {}
    best_loss = 1000000
    for generator_params in generator_grid:
        start = time.time()
        for discriminator_params in discriminator_grid:           
            # Make the models
            generator, generator_optimizer = make_generator_model(**generator_params)
            discriminator, discriminator_optimizer = make_discriminator_model(**discriminator_params)
            
            # Make the loss measure
            cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
            
            # Train it and get the final, total loss
            loss = train(dataset, epochs, cross_entropy,
                         generator, discriminator,
                         generator_optimizer, discriminator_optimizer)
            
            # Store parameters if it is the best
            if(loss < best_loss):
                best_loss = loss
                best_params['generator'] = generator_params
                best_params['discriminator'] = discriminator_params
            count += 1
            
        # Output progress
        print("Done\t" + str(count) + " out of " + str(total) + " | elapsed: " + str(((time.time()-start) / 60)) + "min")
        
    return best_params, best_loss

> Next, let's create the dataset of real images of tents and split them into batches

In [ ]:
# Get all of the real images of tents into numpy arrays
path_preceed = '/content/' if colab else ''
files = glob.glob(path_preceed + "Datasets/xview_tents/xview_tents/*.jpg")
dataset = []
for file in files:
    # Convert to numpy array
    img = PIL.Image.open(file)
    img = img.convert("RGB")
    img = np.asarray(img, dtype=np.float32) / 255
    img = img[:, :, :3]
    dataset.append(img)
    
# Split them into evenly sized batches
batch_size = 2
while(len(dataset) % batch_size != 0):
    batch_size += 1
    
dataset_split = []
for i in range(batch_size, len(dataset) + 1, batch_size):
    dataset_split.append(dataset[i - batch_size : i])
splits = len(dataset_split)
dataset = np.asarray(dataset_split)
    
# Using code from the DCGNN example
dataset = tf.data.Dataset.from_tensor_slices(dataset).shuffle(2020)
print("Using a batch size of " + str(batch_size) + " for a total of " + str(splits) + " splits in the dataset!")

> Next, let's define the parameters to run in the grid search

In [ ]:
# Define the grid search parameters
width = [img.shape[0]]
height = [img.shape[1]]
depth = [img.shape[2]]
activations = [layers.Softmax, layers.LeakyReLU]
optimizers = [tf.keras.optimizers.Adagrad, tf.keras.optimizers.Adam]

# Turn them into dictionaries
generator_grid = dict(width = width,
                    height = height,
                    depth = depth,
                    activation = activations,
                    use_bias = [True, False],
                    optimizer = optimizers)
discriminator_grid = dict(width = width,
                    height = height,
                    depth = depth,
                    activation = activations,
                    dropout = [0.3, 0.4],
                    optimizer = optimizers)

> Finally, let's run the grid search

In [ ]:
best_params, best_loss = gan_grid_search(make_generator_model, make_discriminator_model,
                                         generator_grid, discriminator_grid, 
                                         dataset, batch_size)
print(best_params)
print(best_loss)

> Now, with this configuration, let's train it on the real tents for a much longer duration

In [ ]:
# Make the models
best_params = {'generator': {'activation': layers.LeakyReLU,
   'depth': 3,
   'height': 64,
   'optimizer': tf.keras.optimizers.Adagrad,
   'use_bias': True,
   'width': 64,
    'learning_rate': 1e-3},
  'discriminator': {'activation': layers.LeakyReLU,
   'depth': 3,
   'dropout': 0.4,
   'height': 64,
   'optimizer': tf.keras.optimizers.Adam,
   'width': 64,
   'learning_rate': 1e-3}}

generator, generator_optimizer = make_generator_model(**best_params['generator'])
discriminator, discriminator_optimizer = make_discriminator_model(**best_params['discriminator'])

# Make the loss measure
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Train it and get the final, total loss
loss = train(dataset, 75, cross_entropy,
             generator, discriminator,
             generator_optimizer, discriminator_optimizer)
print("Training complete with a loss of " + str(loss))

> Next, let's generate a set of images

In [ ]:
noise = tf.random.normal([1000, 100])
color_generated_dataset = generator(noise, training=False)

> I have this on my Google Drive so let's download the output
*   https://drive.google.com/file/d/11nLJsGGVCt6nHiv3FqsnNXNcbvdf-v0Y/view?usp=sharing



In [ ]:
!gdown --id 11nLJsGGVCt6nHiv3FqsnNXNcbvdf-v0Y --output dcgan_tents.pkl
import pickle
with open('dcgan_tents.pkl', 'rb') as dcgan_results:
    color_generated_dataset = pickle.load(dcgan_results)
    sess = tf.Session()
    with sess.as_default(): 
      color_generated_dataset = color_generated_dataset.eval()

> Finally, let's view an image from this generation

In [ ]:
plt.imshow(color_generated_dataset[22][:, :, :3])

> We can see that the generation is so much better in quality. I assume this is because of the model having a much lower complexity than that of the StyleGAN2 which is good for our low quanitity + resolution dataset. We will see the performance of the three models on this later on.

In [ ]:
# Save the images to files
def normalize(x):
    return np.array((x - np.min(x)) / (np.max(x) - np.min(x)))

for i in range(len(color_generated_dataset)):
    img = normalize(color_generated_dataset[i][:, :, :3])
    plt.imsave('Datasets/DCGAN/dcgan_tent' + str(i) + '.png', img)

# Model Evaluation: Tents via StyleGAN2

> To properly evaluate the model, I am going to download a dataset I've made on my Google Drive using the past two sections, as well as adding in a variety of different types of features from the xView dataset to make our training/testing dataset complete. The two classes here will be "Tent" or "No Tent".
*    Specifically, I used features classified as "Shed" and "Building". Without any constraints, the xView dataset had 150,329 of these available to me. The average size was below the constraint of 64 x 64 so I just randomly grabbed 1,000 to add to training, and 200 to add to testing
*    In total, we have 2,000 images for training and 418 for testing (roughly an 80/20 split and even distribution of the classes)
*    https://drive.google.com/file/d/1otKjaXNQIOoIkfGpfabNJaorBIkLXAeT/view?usp=sharing

In [ ]:
!gdown --id 1otKjaXNQIOoIkfGpfabNJaorBIkLXAeT --output tentsnet.zip
!unzip -q tentsnet.zip -d tentsnet

> Since the dataset is split into distinct **test** and **train** sets (real and synthetic), we can't do ten cross-fold-validation as we did before. Instead, we will take the accuracy at the end of training for 50 epochs on the dataset in the following method adapted from my validation method before:

In [ ]:
def validate_tent_model(X_train, X_test, Y_train, Y_test,
                        create_cnn_model, params, epochs,
                        outputs_indiv):
    start = time.time()

    # Initialize a model
    if params == None:
        model = create_cnn_model(len(inputs[0]),
                              len(inputs[0][0]),
                              len(inputs[0][0][0]))
    else:
        model = create_cnn_model(**params)

    # Compute the class weights
    weights = class_weight.compute_class_weight('balanced',
                                                np.unique(outputs_indiv), 
                                                outputs_indiv)

    # Train and test the model on this dataset
    result = model.fit(X_train, Y_train,
                        batch_size = 8,
                        epochs = epochs,
                        validation_data = (X_test, Y_test),
                        class_weight = weights,
                        verbose = 0)

    # Print the results and return the model with them
    end = time.time()
    print("Completed in " + str(end - start) + "s" + 
        "\n\tTrain accuracy\t= " + str(result.history['acc'][len(result.history['acc']) - 1]) + 
        "\n\tTrain loss\t= " + str(result.history['loss'][len(result.history['loss']) - 1]) +
        "\n\tTest accuracy\t= " + str(result.history['val_acc'][len(result.history['val_acc']) - 1]) +
        "\n\tTest loss\t= " + str(result.history['val_loss'][len(result.history['val_loss']) - 1]))
    
    return model, result

> Next, lets read each image as a 3D numpy array and store their class
*   I followed the naming convention "class_dataset #.jpg" where class = 0 for no tent or 1 for tent and dataset = test or train

In [ ]:
path_preceed = '/content/' if colab else ''

test_files = glob.glob(path_preceed + "tentsnet/tentsnet/test/*.jpg") + glob.glob(path_preceed + "tentsnet/tentsnet/test/*.png")
test_images = []
test_classes = []
for file in test_files:
  if "0_" in file:
    test_classes.append(0)
  else:
    test_classes.append(1)
  
  # Append the image
  img = PIL.Image.open(file)
  img = np.asarray(img)[:, :, :3]/255
  img = np.pad(img, ((6, 6), (6, 6), (0, 0)),
                   mode='constant', constant_values=3)
  test_images.append(img)

train_files = glob.glob(path_preceed + "tentsnet/tentsnet/train/*.jpg") + glob.glob(path_preceed + "tentsnet/tentsnet/train/*.png")
train_images = []
train_classes = []
for file in train_files:
  if "0_" in file:
    train_classes.append(0)
  else:
    train_classes.append(1)

  # Append the image
  img = PIL.Image.open(file)
  img = np.asarray(img)[:, :, :3]/255
  img = np.pad(img, ((6, 6), (6, 6), (0, 0)),
                   mode='constant', constant_values=3)
  train_images.append(img)

# Convert them all to numpy
test_images = np.asarray(test_images)
test_classes = np.asarray(test_classes)

train_images = np.asarray(train_images)
train_classes = np.asarray(train_classes)

print("Number of training files: " + str(len(train_images)))
print("Number of testing files: " + str(len(test_images)))

> Next, let's convert the outputs to onehot encoding using Keras' to_categorical method


In [ ]:
# Convert the outputs to onehot encoding
from keras.utils import to_categorical
test_outputs_onehot = to_categorical(test_classes)
train_outputs_onehot = to_categorical(train_classes)

---
**Model 1**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_one, verbose=0)

# Define the parameters
width = [train_images[0].shape[0]]
height = [train_images[0].shape[1]]
depth = [train_images[0].shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_one, best_params_one, grid_one = model_grid_search(classifier, grid, 
                                             train_images, train_outputs_onehot)
print("Best score: " + str(best_score_one))
print("Best parameters: " + str(best_params_one))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_one = {'width': train_images[0].shape[0], 'height': train_images[0].shape[1], 'depth': train_images[0].shape[2],
                   'learning_rate': 0.01, 'activation': ReLU, 'optimizer': Adagrad}
model_one = create_cnn_model_one(**best_params_one)
model_one.summary()

> Finally, let's evaluate the optimized model


In [ ]:
validate_tent_model(train_images, test_images, 
                    train_outputs_onehot, test_outputs_onehot,
                    create_cnn_model_one, best_params_one, 10,
                    train_classes)

---
**Model 2**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_two, verbose=0)

# Define the parameters
width = [train_images[0].shape[0]]
height = [train_images[0].shape[1]]
depth = [train_images[0].shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_two, best_params_two, grid_two = model_grid_search(classifier, grid, 
                                             train_images, train_outputs_onehot)
print("Best score: " + str(best_score_two))
print("Best parameters: " + str(best_params_two))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_two = {'width': train_images[0].shape[0], 'height': train_images[0].shape[1], 'depth': train_images[0].shape[2],
                   'learning_rate': 0.01, 'activation': ReLU, 'optimizer': Adagrad}
model_two = create_cnn_model_two(**best_params_two)
model_two.summary()

> Finally, let's evaluate the optimized model


In [ ]:
validate_tent_model(train_images, test_images, 
                    train_outputs_onehot, test_outputs_onehot,
                    create_cnn_model_two, best_params_two, 10,
                    train_classes)

---
**Model 3**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_three, verbose=0)

# Define the parameters
width = [train_images[0].shape[0]]
height = [train_images[0].shape[1]]
depth = [train_images[0].shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_three, best_params_three, grid_three = model_grid_search(classifier, grid, 
                                             train_images, train_outputs_onehot)
print("Best score: " + str(best_score_three))
print("Best parameters: " + str(best_params_three))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_three = {'width': train_images[0].shape[0], 'height': train_images[0].shape[1], 'depth': train_images[0].shape[2],
                   'learning_rate': 0.01, 'activation': Softmax, 'optimizer': Adagrad}
model_three = create_cnn_model_three(**best_params_three)
model_three.summary()

> Finally, we're evaluating this dataset a bit differently than the planes and ships ones since we have a defined test dataset that can't be mixed with the training dataset. So, we'll use the resulting model from the grid search done previously and have it predict the test dataset then show a classification report from sklearn based on the results.


In [ ]:
validate_tent_model(train_images, test_images, 
                    train_outputs_onehot, test_outputs_onehot,
                    create_cnn_model_three, best_params_three, 10,
                    train_classes)

# Model Evaluation: Tents via TensorFlow

> We are going to use the same download source as before for the "tentsnet" dataset and substitute in the generated "color_generated_dataset" from the previous methods.

> Let's read each image as a 3D numpy array and store their class

In [ ]:
path_preceed = '/content/' if colab else ''

test_files = glob.glob(path_preceed + "tentsnet/tentsnet/test/*.jpg") + glob.glob(path_preceed + "tentsnet/tentsnet/test/*.png")
test_images = []
test_classes = []
for file in test_files:
  if "0_" in file:
    test_classes.append(0)
  else:
    test_classes.append(1)
  
  # Add to the dataset
  img = PIL.Image.open(file)
  img = np.asarray(img)[:, :, :3]/255
  img = np.pad(img, ((6, 6), (6, 6), (0, 0)),
                   mode='constant', constant_values=3)
  test_images.append(img)

train_files = glob.glob(path_preceed + "tentsnet/tentsnet/train/*.jpg") + glob.glob(path_preceed + "tentsnet/tentsnet/train/*.png")
train_images = []
train_classes = []

counter = 0
for file in train_files:
  if "0_" in file:
    train_classes.append(0)

    # Add to the dataset
    img = PIL.Image.open(file)
    img = np.asarray(img)[:, :, :3]/255
    img = np.pad(img, ((6, 6), (6, 6), (0, 0)),
                    mode='constant', constant_values=3)
    train_images.append(img)
  else:
    # This is a generated tent; use the color images from the DCGAN model
    train_classes.append(1)
    img = color_generated_dataset[counter][:, :, :3]
    img = np.pad(img, ((6, 6), (6, 6), (0, 0)),
                    mode='constant', constant_values=3)
    train_images.append(img)
    counter += 1

# Convert them all to numpy
test_images = np.asarray(test_images)
test_classes = np.asarray(test_classes)

train_images = np.asarray(train_images)
train_classes = np.asarray(train_classes)

print("Number of training files: " + str(len(train_images)))
print("Number of testing files: " + str(len(test_images)))

> Next, let's convert the outputs to onehot encoding using Keras' to_categorical method


In [ ]:
# Convert the outputs to onehot encoding
from keras.utils import to_categorical
test_outputs_onehot = to_categorical(test_classes)
train_outputs_onehot = to_categorical(train_classes)

---
**Model 1**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_one, verbose=0)

# Define the parameters
width = [train_images[0].shape[0]]
height = [train_images[0].shape[1]]
depth = [train_images[0].shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_one, best_params_one, grid_one = model_grid_search(classifier, grid, 
                                             train_images, train_outputs_onehot)
print("Best score: " + str(best_score_one))
print("Best parameters: " + str(best_params_one))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_one = {'width': train_images[0].shape[0], 'height': train_images[0].shape[1], 'depth': train_images[0].shape[2],
                   'learning_rate': 0.01, 'activation': Softmax, 'optimizer': Adagrad}
model_one = create_cnn_model_one(**best_params_one)
model_one.summary()

> Finally, let's evaluate the optimized model


In [ ]:
validate_tent_model(train_images, test_images, 
                    train_outputs_onehot, test_outputs_onehot,
                    create_cnn_model_one, best_params_one, 10,
                    train_classes)

---
**Model 2**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_two, verbose=0)

# Define the parameters
width = [train_images[0].shape[0]]
height = [train_images[0].shape[1]]
depth = [train_images[0].shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_two, best_params_two, grid_two = model_grid_search(classifier, grid, 
                                             train_images, train_outputs_onehot)
print("Best score: " + str(best_score_two))
print("Best parameters: " + str(best_params_two))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_two = {'width': train_images[0].shape[0], 'height': train_images[0].shape[1], 'depth': train_images[0].shape[2],
                   'learning_rate': 0.01, 'activation': ReLU, 'optimizer': Adagrad}
model_two = create_cnn_model_two(**best_params_two)
model_two.summary()

> Finally, let's evaluate the optimized model


In [ ]:
validate_tent_model(train_images, test_images, 
                    train_outputs_onehot, test_outputs_onehot,
                    create_cnn_model_two, best_params_two, 10,
                    train_classes)

---
**Model 3**

> Next, let's run a grid search on the model to determine the best parameters for this dataset

In [ ]:
# Define the model
classifier = KerasClassifier(build_fn=create_cnn_model_three, verbose=0)

# Define the parameters
width = [train_images[0].shape[0]]
height = [train_images[0].shape[1]]
depth = [train_images[0].shape[2]]
learning_rates = [0.001]
activations = [ReLU, Softmax]
optimizers = [Adagrad, Adam]

# Turn it into a dictionary for the grid
grid = dict(width = width,
            height = height,
            depth = depth,
            learning_rate = learning_rates,
            activation = activations,
            optimizer = optimizers)

# Run the grid search
best_score_three, best_params_three, grid_three = model_grid_search(classifier, grid, 
                                             train_images, train_outputs_onehot)
print("Best score: " + str(best_score_three))
print("Best parameters: " + str(best_params_three))

> Next, let's see what the best model looks like

In [ ]:
# Initialize the best model
best_params_one = {'width': train_images[0].shape[0], 'height': train_images[0].shape[1], 'depth': train_images[0].shape[2],
                   'learning_rate': 0.01, 'activation': ReLU, 'optimizer': Adam}
model_three = create_cnn_model_three(**best_params_three)
model_three.summary()

> Finally, let's evaluate the optimized model


In [ ]:
validate_tent_model(train_images, test_images, 
                    train_outputs_onehot, test_outputs_onehot,
                    create_cnn_model_three, best_params_three, 10,
                    train_classes)